In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash

# Load the CSV data
data_url = 'https://query.data.world/s/q4ux77ari73bdq53esyjjblbuhfchf?dws=00000'
df = pd.read_csv(data_url)

# Create the Dash app
app = JupyterDash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

# Define the threshold value for total_pop and filter the data
threshold = 1000000
filtered_data = df[df['total_pop'] >= threshold]

# Create line plots using Plotly Express
def create_line_plot(data, x_col, y_col, label1, label2):
    fig = px.line(data, x=x_col, y=y_col, color='department_name')
    fig.update_layout(
        #title=f"Relationship of {y_col} to {x_col}",
        xaxis_title=label1,
        yaxis_title=label2,
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        font_family='Arial',
        legend_title_text='Department Name'
    )
    return fig

def create_bar_plot(new_dat):
    fig3 = go.Figure(
    data=[
        go.Bar(
            name="Aggravated Assaults",
            x=new_dat["department_name"],
            y=new_dat["agg_ass_sum"],
            offsetgroup=0
        ),
        go.Bar(
            name="Robberies",
            x=new_dat["department_name"],
            y=new_dat["rob_sum"],
            offsetgroup=1
        ),
        go.Bar(
            name="Rapes",
            x=new_dat["department_name"],
            y=new_dat["rape_sum"],
            offsetgroup=2
        ),
        go.Bar(
            name="Homicides",
            x=new_dat["department_name"],
            y=new_dat["homs_sum"],
            offsetgroup=3
        )
    ],
    layout=go.Layout(
        yaxis_title="Count"
        )
    )
    fig3.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        font_family='Arial',
        legend_title_text='Department Name'
    )
    return fig3

# Create app layout
app.layout = html.Div(style={'backgroundColor': colors['background']},children=[
    html.H1("Crime Analysis for Metropolitan Areas with a Population of 1000000+ Inhabitants (1975-2015)", style={'textAlign': 'center', 'color': colors['text']}),
    
    html.Div([
        html.H3("Data Filtering", style={'textAlign': 'center', 'background-color': colors['background'],'color': colors['text']}),
        dcc.Dropdown(
            id="department-dropdown",
            style={'background-color': colors['background'],'color': '#111111'},
            options=sorted([{"label": dept, "value": dept} for dept in filtered_data['department_name'].unique()],key = lambda x: x['label']),
            value=[dept for dept in filtered_data['department_name'].unique()],
            multi=True
        )
    ]),
    
    html.Div([
        html.H3("Total Violent Crime by Year", style={'textAlign': 'center', 'color': colors['text']}),
        dcc.Graph(id="total-vc-plot")
    ]),
    
    html.Div([
        html.H3("Violent Crimes per 100K Residents by Year", style={'textAlign': 'center', 'color': colors['text']}),
        dcc.Graph(id="vc-plot")
    ]),
    
    html.Div([
        html.H3("Number of Homicides by Year", style={'textAlign': 'center', 'color': colors['text']}),
        dcc.Graph(id="homicide-plot")
    ]),
    
    html.Div([
        html.H3("Violent Crime Type Comparison for 2015", style={'textAlign': 'center', 'color': colors['text']}),
        dcc.Graph(id="crime-comparison-plot")
    ]),
])

# Define app callbacks
@app.callback(
    Output("total-vc-plot", "figure"),
    Output("vc-plot", "figure"),
    Output("homicide-plot", "figure"),
    Output("crime-comparison-plot", "figure"),
    Input("department-dropdown", "value")
)
def update_plots(department_names):
    filtered_data_dept = filtered_data[filtered_data['department_name'].isin(department_names)]
    
    total_vc_plot = create_line_plot(filtered_data_dept, "year", "violent_crime", "Year", "Number of Violent Crimes")
    vc_plot = create_line_plot(filtered_data_dept, "year", "violent_per_100k", "Year", "Number of Violent Crimes per 100K Residents")
    homicide_plot = create_line_plot(filtered_data_dept, "year", "homs_sum", "Year", "Number of Homicides")
    
    double_filt = filtered_data_dept[filtered_data_dept['year']==2015]
    crime_summary_plot = create_bar_plot(double_filt)
    
    return total_vc_plot, vc_plot, homicide_plot, crime_summary_plot

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, host='localhost',port=8052)

Dash app running on http://localhost:8052/
